In [1]:
%run local_functions.py
from local_functions import *


import numpy as np
import math
import glob

import IPython.display as ipd

import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import torch
from torch import nn
import matplotlib.pyplot as plt


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


from scipy.special import expit, logit

import os
import ffmpeg
from math import ceil

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

from datasets import Dataset, DatasetDict
from transformers import EvalPrediction
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification


plt.style.use("dark_background")

pd.set_option("display.max_columns", 2500)
pd.set_option("display.max_rows", 50)

plt.style.use("dark_background")

%load_ext lab_black

2023-09-12 14:09:30.928585: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-12 14:09:31.079090: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 14:09:31.680068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Split into 5 sec clips

In [6]:
audio_files = glob.glob("audio_files/**/*.mp3", recursive=True)

In [13]:

def split_audio(input_file, output_dir, clip_duration=5):
    input_file = os.path.abspath(input_file)
    output_dir = os.path.abspath(output_dir)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get the duration of the input audio file
    probe = ffmpeg.probe(
        input_file, v="error", select_streams="a:0", show_entries="format=duration"
    )
    duration = float(probe["format"]["duration"])

    # Calculate the number of clips needed
    num_clips = ceil(duration / clip_duration)

    # Get the base filename without extension
    base_filename = os.path.splitext(os.path.basename(input_file))[0]

    # Split the audio into clips
    for i in range(num_clips):
        start_time = i * clip_duration
        end_time = min((i + 1) * clip_duration, duration)

        # Generate a unique filename for each clip
        clip_filename = (
            f"{base_filename}_clip_{i + 1}.mp3"  # Change the extension if needed
        )
        output_file = os.path.join(output_dir, clip_filename)

        ffmpeg.input(input_file, ss=start_time, to=end_time).output(output_file).run(
            overwrite_output=True
        )

    print(f"{input_file} split into {num_clips} clips")

In [16]:
# for file in audio_files:
#    artist = file.split("/")[1].split()[0]

#    if __name__ == "__main__":
#        input_audio = file  # Replace with your input audio file
#        output_directory = os.path.join(
#            "split_train_files", artist
#        )  # Construct the output directory path

#       split_audio(input_audio, output_directory)

# Build and Train 

In [2]:
files = get_random_files_from_subdirs("split_train_files", num_files=10)
labels = [file.split()[0].split("/")[-1] for file in files]
df = pd.DataFrame({"file": files, "label": labels})

In [3]:
df["features"] = df["file"].apply(file_to_librosa_features, args=(160000,))
df.drop("file", axis=1, inplace=True)

In [4]:
dummy_df = pd.get_dummies(df["label"], columns=["label"], prefix="", prefix_sep="")
dummy_df = dummy_df.astype(bool)
df = pd.concat([df["features"], dummy_df], axis=1)

In [5]:
df.features[0].shape

(800000,)

In [6]:
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert the split DataFrames into Datasets
train = Dataset.from_pandas(train_df, split="train")
valid = Dataset.from_pandas(valid_df, split="validation")
test = Dataset.from_pandas(test_df, split="test")

dataset = DatasetDict({"train": train, "validation": valid, "test": test})

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['features', 'Crochet', 'Gould', 'Horowitz', 'Ishizaka', 'Moravec', 'Nikolayeva', 'Pogorelich', 'Richter', 'Rubinstein', 'Schiff', 'Tharaud', 'Tureck', '__index_level_0__'],
        num_rows: 84
    })
    validation: Dataset({
        features: ['features', 'Crochet', 'Gould', 'Horowitz', 'Ishizaka', 'Moravec', 'Nikolayeva', 'Pogorelich', 'Richter', 'Rubinstein', 'Schiff', 'Tharaud', 'Tureck', '__index_level_0__'],
        num_rows: 18
    })
    test: Dataset({
        features: ['features', 'Crochet', 'Gould', 'Horowitz', 'Ishizaka', 'Moravec', 'Nikolayeva', 'Pogorelich', 'Richter', 'Rubinstein', 'Schiff', 'Tharaud', 'Tureck', '__index_level_0__'],
        num_rows: 18
    })
})

In [8]:
labels = [
    label
    for label in dataset["train"].features.keys()
    if label not in ["features", "__index_level_0__"]
]
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

In [9]:
extractor = AutoFeatureExtractor.from_pretrained(
    "MIT/ast-finetuned-audioset-10-10-0.4593", max_length=800000
)

In [10]:
extractor

ASTFeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "ASTFeatureExtractor",
  "feature_size": 1,
  "max_length": 800000,
  "mean": -4.2677393,
  "num_mel_bins": 128,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000,
  "std": 4.5689974
}

In [11]:
extractor(
    dataset["train"][0]["features"], sampling_rate=16000, return_tensors="pt"
).input_values.shape

torch.Size([1, 800000, 128])

In [12]:
def preprocess_data(examples):
    # take a batch of texts
    features = examples["features"]
    # encode them
    encoding = extractor(features, sampling_rate=16000, return_tensors="pt")
    # add labels
    # labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    # labels_matrix = np.zeros((len(features), len(labels)))
    # fill numpy array
    # for idx, label in enumerate(labels):
    #    labels_matrix[:, idx] = labels_batch[label]

    # encoding["labels"] = labels_matrix.tolist()

    return encoding

In [66]:
did = preprocess_data(dataset["train"][1])

In [13]:
encoded_dataset = dataset.map(
    preprocess_data, batched=False, remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

In [15]:
encoded_dataset.set_format("torch")

In [43]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values'],
        num_rows: 840
    })
    validation: Dataset({
        features: ['input_values'],
        num_rows: 180
    })
    test: Dataset({
        features: ['input_values'],
        num_rows: 180
    })
})

In [53]:
encoded_dataset["train"]["input_values"][3].shape

torch.Size([1, 1024, 128])

In [44]:
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "MIT/ast-finetuned-audioset-10-10-0.4593",
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([12]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([12, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
batch_size = 4

args = TrainingArguments(
    f"ast-finetuned-ks",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-8,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push_to_hub=True,
)

In [50]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average="micro")
    roc_auc = roc_auc_score(y_true, y_pred, average="micro")
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {"f1": f1_micro_average, "roc_auc": roc_auc, "accuracy": accuracy}
    return metrics


def compute_metrics_2(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids)
    return result

In [51]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=extractor,
    compute_metrics=compute_metrics_2,
)

In [52]:
trainer.train()

  0%|          | 0/52 [00:00<?, ?it/s]

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [4, 1, 1024, 1, 128]

In [38]:
trainer.evaluate()

  0%|          | 0/90 [00:00<?, ?it/s]

{'eval_loss': 0.7002573609352112,
 'eval_f1': 0.1608040201005025,
 'eval_roc_auc': 0.5348484848484848,
 'eval_accuracy': 0.0,
 'eval_runtime': 7.8671,
 'eval_samples_per_second': 22.88,
 'eval_steps_per_second': 11.44,
 'epoch': 5.0}

# Eval 

In [30]:
model = AutoModelForAudioClassification.from_pretrained(
    "ast-finetuned-ks/checkpoint-525",
    num_labels=len(id2label),
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

extractor = AutoFeatureExtractor.from_pretrained(
    "MIT/ast-finetuned-audioset-10-10-0.4593"
)

trainer = Trainer(model=model, tokenizer=extractor)

In [31]:
files = get_random_files_from_subdirs("split_train_files", num_files=100)
labels = [file.split()[0].split("/")[-1] for file in files]
df_test = pd.DataFrame({"file": files, "label": labels})
df_test["features"] = df_test["file"].apply(file_to_librosa_features)
df_test.drop("file", axis=1, inplace=True)

df_test = df_test.sample(frac=1).reset_index(drop=True)

In [32]:
df_test

,label,features
0,Gould,"[-4.8818765e-07, -9.2718983e-07, -6.4910273e-0..."
1,Schiff,"[-0.008452152, -0.017701644, -0.018430043, -0...."
2,Horowitz,"[-7.096678e-07, -1.4826655e-06, -1.5776604e-06..."
3,Richter,"[-1.6298145e-09, 1.3969839e-09, -1.7462298e-09..."
4,Crochet,"[-2.7939677e-09, -1.3969839e-09, -1.3969839e-0..."
...,...,...
1195,Pogorelich,"[-5.005859e-09, -6.4028427e-09, -2.3283064e-09..."
1196,Pogorelich,"[-9.313226e-10, -1.2805685e-09, -5.820766e-10,..."
1197,Gould,"[1.7369166e-07, 3.091991e-07, 2.5704503e-07, 2..."
1198,Schiff,"[-1.6778418e-10, -1.03521386e-10, -3.929264e-1..."


In [33]:
def inference(initial_features, trainer, extractor, id2label, CONFIDENCE_THRESHOLD=0.5):
    encoding = extractor(initial_features, sampling_rate=16000, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

    outputs = trainer.model(**encoding)

    logits = outputs.logits

    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)

    predictions[np.where(probs >= CONFIDENCE_THRESHOLD)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [
        id2label[idx] for idx, label in enumerate(predictions) if label == 1.0
    ]

    return predicted_labels

In [34]:
ROWS_TO_EVALUATE = 500
CONFIDENCE_THRESHOLD = 0.6

df_test["predicted_label"] = df_test["features"][0:ROWS_TO_EVALUATE].apply(
    inference, args=(trainer, extractor, id2label, CONFIDENCE_THRESHOLD)
)

df_test["correct"] = df_test.apply(
    lambda row: int(str(row["label"]) in str(row["predicted_label"])), axis=1
)

df_test["top_n_preds"] = df_test["features"][0:ROWS_TO_EVALUATE].apply(
    n_most_likely_classes, args=(trainer, extractor, id2label, 5)
)

In [35]:
df_test[0:ROWS_TO_EVALUATE]

,label,features,predicted_label,correct,top_n_preds
0,Gould,"[-4.8818765e-07, -9.2718983e-07, -6.4910273e-0...","[Horowitz, Richter, Tureck]",0,"{'Tureck': 0.654, 'Richter': 0.639, 'Horowitz'..."
1,Schiff,"[-0.008452152, -0.017701644, -0.018430043, -0....","[Horowitz, Tureck]",0,"{'Horowitz': 0.661, 'Tureck': 0.611, 'Schiff':..."
2,Horowitz,"[-7.096678e-07, -1.4826655e-06, -1.5776604e-06...","[Horowitz, Ishizaka]",1,"{'Horowitz': 0.699, 'Ishizaka': 0.657, 'Richte..."
3,Richter,"[-1.6298145e-09, 1.3969839e-09, -1.7462298e-09...",[Horowitz],0,"{'Horowitz': 0.753, 'Tureck': 0.57, 'Richter':..."
4,Crochet,"[-2.7939677e-09, -1.3969839e-09, -1.3969839e-0...","[Richter, Tureck]",0,"{'Tureck': 0.638, 'Richter': 0.621, 'Horowitz'..."
...,...,...,...,...,...
495,Richter,"[-8.940697e-08, 2.6077032e-08, -4.7683716e-07,...",[Horowitz],0,"{'Horowitz': 0.674, 'Tureck': 0.6, 'Ishizaka':..."
496,Gould,"[-1.6746654e-10, -7.866863e-11, -1.3521877e-10...","[Horowitz, Tureck]",0,"{'Tureck': 0.666, 'Horowitz': 0.652, 'Ishizaka..."
497,Tureck,"[-3.434252e-08, -1.14087015e-08, -3.934838e-08...",[Richter],0,"{'Richter': 0.615, 'Moravec': 0.581, 'Ishizaka..."
498,Gould,"[-2.0256266e-07, -3.259629e-07, -2.9057264e-07...","[Horowitz, Ishizaka, Tureck]",0,"{'Horowitz': 0.644, 'Tureck': 0.64, 'Ishizaka'..."


In [36]:
df_test[0:ROWS_TO_EVALUATE].correct.sum() / ROWS_TO_EVALUATE

0.18